In [ ]:
import os 
os.chdir("../")

In [ ]:
from __future__ import annotations
import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm

from collections.abc import Iterable


import matplotlib.pyplot as plt

import torch
import ctypes

from dataclasses import dataclass
from typing import Optional, Union
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import (
    PreTrainedTokenizerBase,
    PaddingStrategy,
)
from torch.utils.data import DataLoader
import warnings

warnings.filterwarnings("ignore")


from datasets import load_dataset, load_from_disk
from datasets import concatenate_datasets
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import gc
from transformers import LongformerTokenizer, LongformerForMultipleChoice
import transformers
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import unicodedata
import gc
import pickle
import os


In [ ]:
from src.constants import DATA_PATH,STOP_WORDS,REMOVE_COLS_INFERENCE
from src.params import  MAX_INPUT_INFERENCE,INFERENCE_MODEL_PATH1,INFERENCE_MODEL_PATH2,INFERENCE_MODEL_PATH3
stop_words = STOP_WORDS
MAX_INPUT = MAX_INPUT_INFERENCE


In [ ]:
from src.inference_utis import  SplitList,get_relevant_documents_parsed,get_relevant_documents,retrieval
from src.inference_utis import  softmax
from src.metrics import competition_score
from src.utils import preprocess_wrapper, DataCollatorForMultipleChoice
from src.constants import OPTIONS,INDICES
from src.utils import preprocess_wrapper, DataCollatorForMultipleChoice



In [ ]:
df_valid = pd.read_csv(os.path.join(DATA_PATH,"test.csv"))
retrieved_articles_parsed = get_relevant_documents_parsed(df_valid)
gc.collect()
data_ = retrieved_articles_parsed
contexts = np.array(
    [
        "\n".join([data_[i][e][2] for e in range(len(data_[0]))])
        for i in range(len(data_))
    ]
)
df_valid["context"] = contexts


In [ ]:

test_df = df_valid.copy()
test_df.index = list(range(len(test_df)))
test_df["id"] = list(range(len(test_df)))
# test_df["prompt"] = test_df["context"].apply(lambda x: x[:2000]) + " #### " +  test_df["prompt"]
test_df["answer"] = "A"
test_ds = Dataset.from_pandas(test_df)

r_cols = [
    e
    for e in REMOVE_COLS_INFERENCE
    if e in test_ds.features
]
options = OPTIONS
indices = INDICES



In [ ]:

### DO  PREDICTIONS WITH 3 DIFFERENT MODELS TO ENSEMBLE THEM
from src.utils import create_option_mappings

###### MODEL1 ######
model_dir = INFERENCE_MODEL_PATH1
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

option_to_index,index_to_option = create_option_mappings()

tokenized_test_dataset = test_ds.map(
    preprocess_wrapper(tokenizer, MAX_INPUT=MAX_INPUT),
    batched=False,
    remove_columns=r_cols,
)
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(
    tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator
)
test_predictions1 = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictions1.append(outputs.logits.cpu().detach())

test_predictions1 = torch.cat(test_predictions1)
test_predictions1 = test_predictions1.numpy()




###### MODEL2 ######
model_dir = INFERENCE_MODEL_PATH2
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

test_dataloader = DataLoader(
    tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator
)
test_predictions2 = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictions2.append(outputs.logits.cpu().detach())

test_predictions2 = torch.cat(test_predictions2)
test_predictions2 = test_predictions2.numpy()





###### MODEL3 ######
model_dir = INFERENCE_MODEL_PATH3
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()


test_predictions3 = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictions3.append(outputs.logits.cpu().detach())

test_predictions3 = torch.cat(test_predictions3)
test_predictions3 = test_predictions3.numpy()


In [ ]:
### ensemble predictions
test_predictions = (
    0.2 * softmax(test_predictions1)
    + 0.6 * softmax(test_predictions2)
    + 0.2 * softmax(test_predictions3)
)


predictions_as_ids = np.argsort(-test_predictions, 1)

predictions_as_answer_letters = np.array(list("ABCDE"))[predictions_as_ids]

test_df["prediction"] = [" ".join(row) for row in predictions_as_answer_letters[:, :3]]


submission = test_df[["id", "prediction"]]
#dump predictions
submission.to_csv("submission.csv", index=False)


### prediction scores using labeled dataframe
df = pd.read_csv(os.path.join(DATA_PATH,"train_with_context2.csv"))  # Loading data
test_df["answer"] = df["answer"]
print(competition_score(test_df["answer"], predictions_as_answer_letters))
for pred in [test_predictions1, test_predictions2, test_predictions3]:
    predictions_as_ids = np.argsort(-pred, 1)

    predictions_as_answer_letters = np.array(list("ABCDE"))[predictions_as_ids]

 
    test_df["answer"] = df["answer"]

    print(competition_score(test_df["answer"], predictions_as_answer_letters))